In [1]:
from pathlib import Path
import sys

REPO_ROOT = Path.cwd().resolve()
if (REPO_ROOT / "src").exists() is False:
    # If you opened the notebook from inside /notebooks
    REPO_ROOT = REPO_ROOT.parent

if str(REPO_ROOT) not in sys.path:
    sys.path.insert(0, str(REPO_ROOT))

In [2]:
import sys
from pathlib import Path
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# Make sure repo root is on path (so `import src...` works)
REPO_ROOT = Path.cwd().resolve().parents[0]  # notebooks/ -> repo root
if str(REPO_ROOT) not in sys.path:
    sys.path.insert(0, str(REPO_ROOT))

print("Python executable:", sys.executable)
print("Repo root:", REPO_ROOT)

from src.config import base_case
params = base_case()   # <-- IMPORTANT: call it
from src.model import run_deterministic_unit_econ, result_to_dict

Python executable: c:\Users\Finn Case\OneDrive\Desktop\fintech-unit-econ-sim\.venv\Scripts\python.exe
Repo root: C:\Users\Finn Case\OneDrive\Desktop\fintech-unit-econ-sim


In [3]:
result = run_deterministic_unit_econ(params)
d = result_to_dict(result)

summary = (
    pd.DataFrame([d])
    .T.rename(columns={0: "value"})
)

# Basic numerical sanity checks (won’t catch everything, but catches big mistakes)
assert np.isfinite(summary["value"]).all(), "Found NaN/inf in outputs."

# Rate sanity checks (only check if keys exist)
rate_keys = [
    "default_rate",
    "lgd",
    "apr",
    "funding_cost_rate",
    "take_rate",
]
for k in rate_keys:
    if k in d:
        assert 0 <= d[k] <= 1, f"{k} out of bounds: {d[k]}"

# Common-sense sign checks (only check if keys exist)
# Revenue and costs should be >= 0 in most models
nonneg_keys = [
    "interest_revenue",
    "fee_revenue",
    "funding_cost",
    "servicing_cost",
    "acquisition_cost",
    "expected_credit_loss",
]
for k in nonneg_keys:
    if k in d:
        assert d[k] >= 0, f"{k} should be >= 0, got {d[k]}"

print("✅ Deterministic model ran successfully.")
summary.head(25)

✅ Deterministic model ran successfully.


,value
accounts,2.000000e+04
annual_interest_revenue,9.900000e+06
annual_fee_revenue,2.000000e+06
annual_gross_revenue,1.190000e+07
expected_credit_loss,4.200000e+06
funding_cost,3.300000e+06
servicing_cost,1.000000e+06
acquisition_cost,5.000000e+06
contribution_profit,-1.600000e+06
contribution_margin,-1.344538e-01


In [4]:
print("Revenue per account:", d["annual_gross_revenue"] / d["accounts"])
print("Expected loss per account:", d["expected_credit_loss"] / d["accounts"])
print("CAC per account:", d["acquisition_cost"] / d["accounts"])


Revenue per account: 595.0000000000001
Expected loss per account: 210.0
CAC per account: 250.0
